In [5]:
import pandas as pd
import numpy as np

In [6]:
import datetime

In [7]:
from matplotlib import pyplot as plt
%matplotlib inline


In [8]:
import backtrader as bt


In [17]:
import datetime

In [9]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

In [10]:
import talib 

In [11]:
from futu import *

In [77]:
from jqdatasdk import *
auth('18688709107','chan123456')

In [82]:
import os

In [4]:
start_date = '2019-12-01'
end_date ='2020-09-12'
stock_list =['HK.00700','HK.01810','HK.03690','HK.00285','HK.09988']

In [12]:
def get_stockdata(stock_list,start_date,end_date):
    df={}
    for stock in stock_list:
        
        quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
        ret, stock_data, page_req_key = quote_ctx.request_history_kline(stock, start=start_date, end=end_date)  
        quote_ctx.close() 
        
        
        stock_data['date']=pd.to_datetime(stock_data['time_key'])
        stock_data=stock_data.set_index('date',drop=True)
        stock_data=stock_data.drop(columns=['time_key'])
        
        stock_data['openinterest']=0
        df.update({stock:stock_data})
    return df
    
   
    

In [13]:
df_stock=get_stockdata(stock_list,start_date,end_date)

2020-09-24 03:10:30,473 [open_context_base.py] _socket_reconnect_and_wait_ready:255: Start connecting: host=127.0.0.1; port=11111;
2020-09-24 03:10:30,478 [open_context_base.py] on_connected:344: Connected : conn_id=1; 
2020-09-24 03:10:30,486 [open_context_base.py] _handle_init_connect:445: InitConnect ok: conn_id=1; info={'server_version': 216, 'login_user_id': 245840, 'conn_id': 6714838401075303998, 'conn_key': 'A09D4BD79A91E053', 'conn_iv': 'F0E1DD7D169253ED', 'keep_alive_interval': 10, 'is_encrypt': False};
2020-09-24 03:10:30,584 [network_manager.py] work:366: Close: conn_id=1
2020-09-24 03:10:30,597 [open_context_base.py] _socket_reconnect_and_wait_ready:255: Start connecting: host=127.0.0.1; port=11111;
2020-09-24 03:10:30,599 [open_context_base.py] on_connected:344: Connected : conn_id=2; 
2020-09-24 03:10:30,606 [open_context_base.py] _handle_init_connect:445: InitConnect ok: conn_id=2; info={'server_version': 216, 'login_user_id': 245840, 'conn_id': 6714838401570475531, 'con

In [18]:
#HSI index
quote_ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
ret, HSI_data, page_req_key = quote_ctx.request_history_kline('HK.800000', start=start_date, end=end_date)  
quote_ctx.close() 
        

2020-09-17 20:05:08,759 [open_context_base.py] _socket_reconnect_and_wait_ready:255: Start connecting: host=127.0.0.1; port=11111;
2020-09-17 20:05:08,761 [open_context_base.py] on_connected:344: Connected : conn_id=7; 
2020-09-17 20:05:08,765 [open_context_base.py] _handle_init_connect:445: InitConnect ok: conn_id=7; info={'server_version': 216, 'login_user_id': 245840, 'conn_id': 6712557028037024031, 'conn_key': '3AF0442D8E91541E', 'conn_iv': '9FCFEDC593B1F66D', 'keep_alive_interval': 10, 'is_encrypt': False};
2020-09-17 20:05:08,803 [network_manager.py] work:366: Close: conn_id=7


In [210]:
def get_boll(end_date):
        """
        获取北向资金布林带
        """
        stdev_n = 2
        global df_GGT
        table = finance.STK_ML_QUOTA # 市场通成交与额度信息
        """q = query(
            table.day, table.quota_daily, table.quota_daily_balance
        ).filter(
            table.link_id.in_(['310001', '310002']), table.day<=end_date  #沪股通、深股通
        ).order_by(table.day)
        money_df = finance.run_query(q)
        """

        
        
        """end_date =  datetime.datetime.strptime(end_date ,'%Y-%m-%d')
        print(end_date.strftime('%Y-%m-%d'))"""
        money_df = df_GGT
        print(end_date)
        
        if len(money_df[money_df.day==end_date]) ==0:
            return 'itis not  the tradeday'
        idx = money_df[money_df.day==end_date].index[-1]
        
        
        money_df=money_df.iloc[:idx+1].copy()
        
        money_df['net_amount'] = money_df['quota_daily'] - money_df['quota_daily_balance'] #每日额度-每日剩余额度=净买入额
        print (money_df)
        # 分组求和
        money_df = money_df.groupby('day')[['net_amount']].sum().iloc[-60:] #过去g.window天求和
        
        mid = money_df['net_amount'].mean()
        stdev = money_df['net_amount'].std()
        upper = mid + stdev_n * stdev
        lower = mid - stdev_n * stdev
        mf = money_df['net_amount'].iloc[-1]
        
        
        print(mf,upper,lower)
        if mf > int(upper):
            return 'buy'
                        
        # 净买入额在布林线下轨以下时清仓
        elif  mf <= lower:
            
            return 'sell'
        else:
            return 'other'
        
        

In [14]:
df_GGT=pd.read_csv('北上.csv',index_col=0)

In [18]:
class TXStrategy(bt.Strategy):
    # 设置简单均线周期，以备后面调用
    params = (
        ('maperiod21', 21),
        ('maperiod55', 55)
    )

    def log(self, txt, dt=None):
        # 日记记录输出
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # 初始化数据参数
        # 设置当前收盘价为dataclose
        self.dataclose = self.datas[0].close

        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.signal_1='no'
        self.buylist =[]
        
        self.lastlist =[]
        self.GGT = df_GGT
        self.stocklist=stock_list
        # 添加简单均线
        self.sma21 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod21)
        self.sma55 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod55)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 如果有订单提交或者已经接受的订单，返回退出
            return
        # 主要是检查有没有成交的订单，如果有则日志记录输出价格，金额，手续费。注意，如果资金不足是不会成交订单的
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED ,Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            # len(self)是指获取截至当前数据一共有多少根bar
            # 以下代码就是指当交易发生时立刻记录下了当天有多少根bar
            # 如果要表示当成交后过了5天卖，则可以这样写 if len(self) >= (self.bar_executed + 5):
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:  # 如果交易还没有关闭，则退出不输出显示盈利跟手续费
            return
        self.log(' OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    
      
    
    
    def get_daily_list(self):
        global df_stock
        
        MA=13
        i=0 # 计数器初始化
        # 创建保持计算结果的DataFrame
        df = pd.DataFrame()
        for k in self.stocklist:
            security = k# key()以指数计算交易信号，values()以基金计算交易信号
            # 获取股票的收盘价
            
            close_data = df_stock[security]
            
            # 获取股票现价
            date = self.datas[0].datetime.date(0).isoformat()
            
            idx= close_data.index.get_loc(date)
            close_data = close_data.iloc[idx - MA+1 :idx+1]['close']
            
            current_price = close_data.iloc[-1]                       
            
            cp_increase = (current_price/close_data[0]-1)*100
            
            # 取得平均价格
            ma_n1 = close_data.mean()
            # 计算前一收盘价与均值差值    
            pre_price = (current_price/ma_n1-1)*100
            
            df.loc[i,'股票代码'] = security # 把标的股票代码添加到DataFrame
            df.loc[i,'股票名称'] = 'tx' # 把标的股票名称添加到DataFrame
            df.loc[i,'周期涨幅'] = cp_increase # 把计算结果添加到DataFrame
            df.loc[i,'均线差值'] = pre_price # 把计算结果添加到DataFrame
            i=i+1             
        
        df = df.fillna(-100)
        df.sort_values(by='周期涨幅',ascending=False,inplace=True) # 按照涨幅排序        
        df.reset_index(drop=True, inplace=True) # 重新设置索        
        self.buylist=[]

        
        if df.iloc[0,2] < 0.1 or df.iloc[0,3] < 0: # 均不符合买入条件    
            if len(self.lastlist) == 0:# 持仓为空
                
                return 'KEEP'# 持仓保持不变
                
            else:# 持仓不为空
                               
                return 'sell_the_stocks' 
            
            
        #删除不符合要求的标的
        for t in df.index:
            if df.loc[t,'周期涨幅'] < 0.1 or df.loc[t,'均线差值'] < 0:
                df=df.drop(t)
        # 表不为空,持有排名前两名的股
        
        if df.shape[0] > 2 :
            
            self.buylist.append(str(df.iloc[0,0]))
            self.buylist.append(str(df.iloc[1,0]))
            #self.buylist.append(str(df.iloc[2,0]))
            
            return 'BUY'
        # 表不为空,持有排名前两名的股
        if df.shape[0] > 1 :# 表不为空,持有排名前两名的股
            self.buylist.append(str(df.iloc[0,0]))
            self.buylist.append(str(df.iloc[1,0]))

            return 'BUY'    
        else: # 持第一名的股
            self.buylist.append(str(df.iloc[0,0]))                
            return 'BUY'
        
    def get_boll(self,end_date):
        """
        获取北向资金布林带
        """
        stdev_n = 2
        
        
        '''
        table = finance.STK_ML_QUOTA # 市场通成交与额度信息
        
        q = query(
            table.day, table.quota_daily, table.quota_daily_balance
        ).filter(
            table.link_id.in_(['310001', '310002']), table.day<=end_date  #沪股通、深股通
        ).order_by(table.day)
        
        south = query(
            table.day, table.quota_daily, table.quota_daily_balance
        ).filter(
            table.link_id.in_(['310003', '310004']), table.day<=end_date  #沪股通、深股通
        ).order_by(table.day)
        '''
        
        
        
        money_df = self.GGT
        
        if len(money_df[money_df.day==end_date]) ==0:
            return
        idx = money_df[money_df.day==end_date].index[-1]
        money_df=money_df.loc[:idx].copy()
        
        #money_df = finance.run_query(south)

        money_df['net_amount'] = money_df['quota_daily'] - money_df['quota_daily_balance'] #每日额度-每日剩余额度=净买入额
        # 分组求和
        money_df = money_df.groupby('day')[['net_amount']].sum().iloc[-150:] #过去g.window天求和
        mid = money_df['net_amount'].mean()
        stdev = money_df['net_amount'].std()
        upper = mid + stdev_n * stdev
        lower = mid - stdev_n * stdev
        mf = money_df['net_amount'].iloc[-1]
        print(mf,upper,lower)
        if mf > int(upper):
            return 'buy'
                        
        # 净买入额在布林线下轨以下时清仓
        elif  mf <= lower:
            print('----------------------sell------------')
            return 'sell'
        else:
            return 'no signal'
             
        
        
    def Index_Emotion(self):
        MA_1 = 12
        MA_2 = 6
        ret_val=0
        #volume = attribute_history(security=g.target_market, count=_cnt, unit='1d', fields='volume')['volume']
        date = self.datas[0].datetime.date(0).isoformat()
        
        idx= HSI_data.index.get_loc(date)
                
        volume = HSI_data.iloc[idx-MA_1+1:idx+1]['turnover']        
        
        v_ma = talib.MA(volume, MA_2+1)         
                
        #
        #g.emotion_rate = round((volume[-1] / v_ma_lag0[-1] - 1) * 100, 2)  # 最后一天的成交量比成交量均值高%
        #
        vol_diff = (volume - v_ma)

     
        if vol_diff[-1] >= 0:
            ret_val = 1 if (vol_diff[-3:] >= 0).all() else 0  # 大盘成交量，最近3天都站在均线之上
        else:
            ret_val = -1 if (vol_diff[-MA_2:] < 0).all() else 0  # 大盘成交量，最近6天都在均线之下
                       
        if ret_val <0:
            
            print('------------------------------date',date,'ret_val',ret_val)
            #print(volume)
            #print(v_ma)
            #print(vol_diff)
        return ret_val        
    
       
        
        
    def next(self):
        # 输出显示收盘价
        #self.log('Close, %.2f' % self.dataclose[0])
        #print("name :%s, price:%.2f" % (data._name, data[0]))
        # 检查是否有订单发送当中，如果有则不再发送第二个订单
       
        signal = self.get_daily_list()
        date = self.datas[0].datetime.date(0).isoformat()
        
        
        
        
        if signal == 'sell_the_stocks' :
            self.log("交易信号:空仓")   
            for data in self.datas:
                self.log('卖出 %s '% data._name)
                self.close(data)
               
            
            self.lastlist=[]
            
            
        elif signal == 'KEEP':
            self.log("交易信号:持仓不变")
        
        elif signal == 'BUY':    
            self.log("交易信号:买入")
            clear_list = []
            ratio = len(self.buylist)
            daily_value = self.broker.get_value()
            self.log('daily_value %.2f' % daily_value)
            #self.log('buylist %s ,lastlist %s'%(self.buylist,self.lastlist))
            for list in self.lastlist:
                
                if list not in self.buylist:
                    
                    self.close(list)
                    self.log('卖出 %s' %list)
                    
            for to_buy in self.buylist:
                if to_buy  not in self.lastlist:
                    close_price = df_stock[to_buy].loc[date].close
                    #print('date : ',date,to_buy,close_price)
                    buycash = daily_value/ratio
                    buysize = ((buycash/close_price)//100)*100
                    #print ('buycash',buycash,'buysize',buysize)
                                    
                    self.log('买入 %s' %to_buy)
                    self.order_target_value(to_buy,buycash*0.8)
                    
                    
            #for buy_stock in self.buylist:
                #self.order_target_size(target=1)

        self.lastlist = self.buylist          
                
        


In [19]:
class TotalValue(bt.Analyzer):
    '''This analyzer will get total value from every next.

    Params:
    Methods:

      - get_analysis

        Returns a dictionary with returns as values and the datetime points for
        each return as keys
    '''

    params = ( )

    def start(self):
        super(TotalValue, self).start()
        self.rets = OrderedDict()

    def next(self):
        # Calculate the return
        super(TotalValue, self).next()
        self.rets[self.datas[0].datetime.datetime()] = self.strategy.broker.getvalue()
        
    def get_analysis(self):
        return self.rets

In [20]:
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(TXStrategy)

#data = bt.feeds.PandasData(dataname=TX_data,name='HK.00700',fromdate=datetime.datetime(2019, 12, 1),todate=datetime.datetime(2020, 8, 31))
#data2 = bt.feeds.PandasData(dataname=df_stock['HK.09988'],name='HK.09988',fromdate=datetime.datetime(2019, 12, 1),todate=datetime.datetime(2020, 8, 31))

    # 设置资金
    
for s in stock_list:
    feed = bt.feeds.PandasData(dataname =df_stock[s] ,fromdate=datetime.datetime(2018, 11, 1),todate=datetime.datetime(2020,9, 12))
    cerebro.adddata(feed, name = s)
    
    
    
cerebro.broker.setcash(1000000.0)
    # 设置每笔交易交易的股票数量
cerebro.addsizer(bt.sizers.FixedSize, stake=100)
    # 设置手续费
cerebro.broker.setcommission(commission=0.002)
    # 输出初始资金
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    # 运行策略
cerebro.run()
    # 输出结果
    
#cerebro.addanalyzer('TotalValue')

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

%matplotlib inline
#cerebro.plot()




Starting Portfolio Value: 1000000.00
2020-02-21, 交易信号:买入
2020-02-21, daily_value 1000000.00
2020-02-21, 买入 HK.00285
2020-02-21, 买入 HK.03690
2020-02-24, BUY EXECUTED ,Price: 15.94, Cost: 390680.29, Comm 781.36
2020-02-24, BUY EXECUTED ,Price: 101.50, Cost: 393312.50, Comm 786.62
2020-02-24, 交易信号:空仓
2020-02-24, 卖出 HK.00700 
2020-02-24, 卖出 HK.01810 
2020-02-24, 卖出 HK.03690 
2020-02-24, 卖出 HK.00285 
2020-02-24, 卖出 HK.09988 
2020-02-25, SELL EXECUTED, Price: 99.80, Cost: 393312.50, Comm 773.45
2020-02-25, SELL EXECUTED, Price: 15.30, Cost: 390680.29, Comm 749.99
2020-02-25,  OPERATION PROFIT, GROSS -6587.50, NET -8147.58
2020-02-25,  OPERATION PROFIT, GROSS -15683.84, NET -17215.19
2020-02-25, 交易信号:持仓不变
2020-02-26, 交易信号:买入
2020-02-26, daily_value 974637.23
2020-02-26, 买入 HK.03690
2020-02-27, BUY EXECUTED ,Price: 101.70, Cost: 777394.80, Comm 1554.79
2020-02-27, 交易信号:买入
2020-02-27, daily_value 985312.84
2020-02-27, 买入 HK.01810
2020-02-28, Order Canceled/Margin/Rejected
2020-02-28, 交易信号:空仓
20

In [241]:
class reblanc_Strategy(bt.Strategy):
    # 设置简单均线周期，以备后面调用
    params = (
        ('maperiod21', 21),
        ('maperiod55', 55)
    )

    def log(self, txt, dt=None):
        # 日记记录输出
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # 初始化数据参数
        # 设置当前收盘价为dataclose
        self.dataclose = self.datas[0].close

        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.signal_1='no'
        self.buylist =[]
        self.lastlist =[]
        self.signaltag = None 
        self.GGT = df_GGT
        self.stocklist=stock_list
        # 添加简单均线
        self.sma21 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod21)
        self.sma55 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod55)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # 如果有订单提交或者已经接受的订单，返回退出
            return
        # 主要是检查有没有成交的订单，如果有则日志记录输出价格，金额，手续费。注意，如果资金不足是不会成交订单的
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED ,Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
            # len(self)是指获取截至当前数据一共有多少根bar
            # 以下代码就是指当交易发生时立刻记录下了当天有多少根bar
            # 如果要表示当成交后过了5天卖，则可以这样写 if len(self) >= (self.bar_executed + 5):
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:  # 如果交易还没有关闭，则退出不输出显示盈利跟手续费
            return
        self.log(' OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
    
      
    
    
    def get_daily_list(self):
        global df_stock
        
        MA=13
        i=0 # 计数器初始化
        # 创建保持计算结果的DataFrame
        df = pd.DataFrame()
        for k in self.stocklist:
            security = k# key()以指数计算交易信号，values()以基金计算交易信号
            # 获取股票的收盘价
            
            close_data = df_stock[security]
            
            # 获取股票现价
            date = self.datas[0].datetime.date(0).isoformat()
            
            idx= close_data.index.get_loc(date)
            close_data = close_data.iloc[idx - MA+1 :idx+1]['close']
            
            current_price = close_data.iloc[-1]                       
            
            cp_increase = (current_price/close_data[0]-1)*100
            
            # 取得平均价格
            ma_n1 = close_data.mean()
            # 计算前一收盘价与均值差值    
            pre_price = (current_price/ma_n1-1)*100
            
            df.loc[i,'股票代码'] = security # 把标的股票代码添加到DataFrame
            df.loc[i,'股票名称'] = 'tx' # 把标的股票名称添加到DataFrame
            df.loc[i,'周期涨幅'] = cp_increase # 把计算结果添加到DataFrame
            df.loc[i,'均线差值'] = pre_price # 把计算结果添加到DataFrame
            i=i+1             
        
        df = df.fillna(-100)
        df.sort_values(by='周期涨幅',ascending=False,inplace=True) # 按照涨幅排序        
        df.reset_index(drop=True, inplace=True) # 重新设置索        
        self.buylist=[]
        
        
        if df.iloc[0,2] < 0.1 or df.iloc[0,3] < 0: # 均不符合买入条件    
            if len(self.lastlist) == 0:# 持仓为空
                
                return 'KEEP'# 持仓保持不变
                
            else:# 持仓不为空
                               
                return 'sell_the_stocks' 
            
            
        #删除不符合要求的标的
        for t in df.index:
            if df.loc[t,'周期涨幅'] < 0.1 or df.loc[t,'均线差值'] < 0:
                df=df.drop(t)
        # 表不为空,持有排名前两名的股
        
        if df.shape[0] > 2 :
            
            self.buylist.append(str(df.iloc[0,0]))
            self.buylist.append(str(df.iloc[1,0]))
            #self.buylist.append(str(df.iloc[2,0]))
            
            return 'BUY'
        # 表不为空,持有排名前两名的股
        if df.shape[0] > 1 :# 表不为空,持有排名前两名的股
            self.buylist.append(str(df.iloc[0,0]))
            self.buylist.append(str(df.iloc[1,0]))

            return 'BUY'    
        else: # 持第一名的股
            self.buylist.append(str(df.iloc[0,0]))                
            return 'BUY'
        
    def get_boll(self,end_date):
        """
        获取北向资金布林带
        """
        stdev_n = 2
        
        
        '''
        table = finance.STK_ML_QUOTA # 市场通成交与额度信息
        
        q = query(
            table.day, table.quota_daily, table.quota_daily_balance
        ).filter(
            table.link_id.in_(['310001', '310002']), table.day<=end_date  #沪股通、深股通
        ).order_by(table.day)
        
        south = query(
            table.day, table.quota_daily, table.quota_daily_balance
        ).filter(
            table.link_id.in_(['310003', '310004']), table.day<=end_date  #沪股通、深股通
        ).order_by(table.day)
        '''
        
        
        
        money_df = self.GGT
        
        if len(money_df[money_df.day==end_date]) ==0:
            return
        idx = money_df[money_df.day==end_date].index[-1]
        money_df=money_df.loc[:idx].copy()
        
        #money_df = finance.run_query(south)

        money_df['net_amount'] = money_df['quota_daily'] - money_df['quota_daily_balance'] #每日额度-每日剩余额度=净买入额
        # 分组求和
        money_df = money_df.groupby('day')[['net_amount']].sum().iloc[-150:] #过去g.window天求和
        mid = money_df['net_amount'].mean()
        stdev = money_df['net_amount'].std()
        upper = mid + stdev_n * stdev
        lower = mid - stdev_n * stdev
        mf = money_df['net_amount'].iloc[-1]
        print(mf,upper,lower)
        if mf >=int(upper):
            self.signaltag ='UPCROSS'
            return 'up'
                        
        # 净买入额在布林线下轨以下时清仓
        elif  mf <= lower:
            print('----------------------sell------------')
            self.signaltag ='DOWNCROSS'
            return 'down'
        else:
            return 'no signal'
             
        
        
    def Index_Emotion(self):
        MA_1 = 12
        MA_2 = 6
        ret_val=0
        #volume = attribute_history(security=g.target_market, count=_cnt, unit='1d', fields='volume')['volume']
        date = self.datas[0].datetime.date(0).isoformat()
        
        idx= HSI_data.index.get_loc(date)
                
        volume = HSI_data.iloc[idx-MA_1+1:idx+1]['turnover']        
        
        v_ma = talib.MA(volume, MA_2+1)         
                
        #
        #g.emotion_rate = round((volume[-1] / v_ma_lag0[-1] - 1) * 100, 2)  # 最后一天的成交量比成交量均值高%
        #
        vol_diff = (volume - v_ma)

     
        if vol_diff[-1] >= 0:
            ret_val = 1 if (vol_diff[-3:] >= 0).all() else 0  # 大盘成交量，最近3天都站在均线之上
        else:
            ret_val = -1 if (vol_diff[-MA_2:] < 0).all() else 0  # 大盘成交量，最近6天都在均线之下
                       
        if ret_val <0:
            
            print('------------------------------date',date,'ret_val',ret_val)
            #print(volume)
            #print(v_ma)
            #print(vol_diff)
        return ret_val        
    
       
        
        
    def next(self):
        # 输出显示收盘价
        #self.log('Close, %.2f' % self.dataclose[0])
        #print("name :%s, price:%.2f" % (data._name, data[0]))
        # 检查是否有订单发送当中，如果有则不再发送第二个订单
       
        signal = self.get_daily_list()
        date = self.datas[0].datetime.date(0).isoformat()
        
        tradesignal = self.get_boll(date)
        
        if tradesignal == 'down':

            self.log("交易信号:空仓")   
            for data in self.datas:
                self.log('卖出 %s '% data._name)
                self.close(data)    
            
            self.lastlist=[]
        elif tradesignal == 'up':
            self.log("交易信号:买入")

            clear_list = []
            ratio = len(self.buylist)
            daily_value = self.broker.get_value()
            self.log('daily_value %.2f' % daily_value)
            #self.log('buylist %s ,lastlist %s'%(self.buylist,self.lastlist))
            if self.buylist ==[]:
                for list in self.lastlist:
                    self.close(list)
                    self.log('卖出 %s' %list)
            
            for list in self.lastlist:
                
                if list not in self.buylist:
                    
                    self.close(list)
                    self.log('卖出 %s' %list)
                    
            for to_buy in self.buylist:
                if to_buy  not in self.lastlist:
                    close_price = df_stock[to_buy].loc[date].close
                    #print('date : ',date,to_buy,close_price)
                    buycash = daily_value/ratio
                    buysize = ((buycash/close_price)//100)*100
                    #print ('buycash',buycash,'buysize',buysize)
                                    
                    self.log('买入 %s' %to_buy)
                    self.order_target_value(to_buy,buycash*0.8)
        elif self.signaltag =='UPCROSS':
            self.log("交易信号:持有 up")

            clear_list = []
            ratio = len(self.buylist)
            daily_value = self.broker.get_value()
            self.log('daily_value %.2f' % daily_value)
            #self.log('buylist %s ,lastlist %s'%(self.buylist,self.lastlist))
            if self.buylist ==[]:
                for list in self.lastlist:
                    self.close(list)
                    self.log('卖出 %s' %list)
            
            for list in self.lastlist:
                
                if list not in self.buylist:
                    
                    self.close(list)
                    self.log('卖出 %s' %list)
                    
            for to_buy in self.buylist:
                if to_buy  not in self.lastlist:
                    close_price = df_stock[to_buy].loc[date].close
                    #print('date : ',date,to_buy,close_price)
                    buycash = daily_value/ratio
                    buysize = ((buycash/close_price)//100)*100
                    #print ('buycash',buycash,'buysize',buysize)
                                    
                    self.log('买入 %s' %to_buy)
                    self.order_target_value(to_buy,buycash*0.8)
            
        else:
                self.log("交易信号: down cross")

        self.lastlist = self.buylist          
                
        


In [256]:
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(reblanc_Strategy)

#data = bt.feeds.PandasData(dataname=TX_data,name='HK.00700',fromdate=datetime.datetime(2019, 12, 1),todate=datetime.datetime(2020, 8, 31))
#data2 = bt.feeds.PandasData(dataname=df_stock['HK.09988'],name='HK.09988',fromdate=datetime.datetime(2019, 12, 1),todate=datetime.datetime(2020, 8, 31))

    # 设置资金
    
for s in stock_list:
    feed = bt.feeds.PandasData(dataname =df_stock[s] ,fromdate=datetime.datetime(2019, 11, 1),todate=datetime.datetime(2020,9, 12))
    cerebro.adddata(feed, name = s)
    
    
    
cerebro.broker.setcash(1000000.0)
    # 设置每笔交易交易的股票数量
cerebro.addsizer(bt.sizers.FixedSize, stake=100)
    # 设置手续费
cerebro.broker.setcommission(commission=0.002)
    # 输出初始资金
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    # 运行策略
cerebro.run()
    # 输出结果
    
#cerebro.addanalyzer('TotalValue')

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

%matplotlib inline
#cerebro.plot()




Starting Portfolio Value: 1000000.00
-48.5068 125.03935572991446 -28.144909063247802
----------------------sell------------
2020-01-21, 交易信号:空仓
2020-01-21, 卖出 HK.00700 
2020-01-21, 卖出 HK.01810 
2020-01-21, 卖出 HK.03690 
2020-01-21, 卖出 HK.00285 
2020-01-21, 卖出 HK.09988 
17.69440000000003 124.98044815701687 -27.931051490350193
2020-01-22, 交易信号: down cross
-94.30629999999996 131.24136772068144 -38.162401054014744
----------------------sell------------
2020-01-23, 交易信号:空仓
2020-01-23, 卖出 HK.00700 
2020-01-23, 卖出 HK.01810 
2020-01-23, 卖出 HK.03690 
2020-01-23, 卖出 HK.00285 
2020-01-23, 卖出 HK.09988 
2020-01-24, 交易信号: down cross
2020-01-29, 交易信号: down cross
2020-01-30, 交易信号: down cross
2020-01-31, 交易信号: down cross
199.32260000000002 142.63053049520755 -44.143970495207554
2020-02-03, 交易信号:买入
2020-02-03, daily_value 1000000.00
79.87400000000002 143.6476396054689 -43.558772938802214
2020-02-04, 交易信号:持有 up
2020-02-04, daily_value 1000000.00
24.92029999999994 143.6106363426599 -43.93166967599321
2020-

In [405]:
#### 海龟策略
class TestSizer(bt.Sizer):
    params = (('stake', 1),)    
    def _getsizing(self, comminfo, cash, data, isbuy):        
        if isbuy:          
            return self.p.stake        
        position = self.broker.getposition(data)        
        if not position.size:            
            return 0        
        else:            
            return position.size        
        return self.p.stakeclass 

class TestStrategy(bt.Strategy):
    params = ( ('maperiod', 30),  ('printlog', False), )   
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:            
            dt = dt or self.datas[0].datetime.date(0)           
            print('%s, %s' % (dt.isoformat(), txt))    

    def __init__(self):        

        self.dataclose = self.datas[0].close      
        self.datahigh = self.datas[0].high        
        self.datalow = self.datas[0].low     

        self.order = None      
        self.buyprice = 0      
        self.buycomm = 0      
        self.newstake = 0      
        self.buytime = 0       
        # 参数计算，唐奇安通道上轨、唐奇安通道下轨、ATR        
        self.DonchianHi = bt.indicators.Highest(self.datahigh(-1), period=20, subplot=False)        
        self.DonchianLo = bt.indicators.Lowest(self.datalow(-1), period=10, subplot=False)       
        self.TR = bt.indicators.Max((self.datahigh(0)- self.datalow(0)), abs(self.dataclose(-1) -   self.datahigh(0)), abs(self.dataclose(-1)  - self.datalow(0) ))        
        self.ATR = bt.indicators.SimpleMovingAverage(self.TR, period=14, subplot=True)       
        # 唐奇安通道上轨突破、唐奇安通道下轨突破       
        self.CrossoverHi = bt.ind.CrossOver(self.dataclose(0), self.DonchianHi)        
        self.CrossoverLo = bt.ind.CrossOver(self.dataclose(0), self.DonchianLo)    
    def notify_order(self, order):        
        if order.status in [order.Submitted, order.Accepted]:            
            return

        if order.status in [order.Completed]:            
            if order.isbuy():               
                self.log(                    
                'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %                   
                (order.executed.price,
                order.executed.value,
                order.executed.comm),doprint=True)              
                self.buyprice = order.executed.price              
                self.buycomm = order.executed.comm            
            else:             
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %                        
                     (order.executed.price,
                     order.executed.value,
                     order.executed.comm),doprint=True)
                self.bar_executed = len(self)       
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:           
            self.log('Order Canceled/Margin/Rejected')        
        self.order = None    

    def notify_trade(self, trade):      
        if not trade.isclosed:
            return        
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' % (trade.pnl, trade.pnlcomm)) 

    def next(self): 
        if self.order:
            return        
        #入场        
        if self.CrossoverHi > 0 and self.buytime == 0:                                 
            self.newstake = self.broker.getvalue() * 0.01 / self.ATR            
            self.newstake = int(self.newstake / 100) * 100                             
            self.sizer.p.stake = self.newstake            
            self.buytime = 1            
            self.order = self.buy()        
        #加仓        
        elif self.datas[0].close >self.buyprice+0.5*self.ATR[0] and self.buytime > 0 and self.buytime < 5:           
            self.newstake = self.broker.getvalue() * 0.01 / self.ATR            
            self.newstake = int(self.newstake / 100) * 100            
            self.sizer.p.stake = self.newstake            
            self.order = self.buy()           
            self.buytime = self.buytime + 1        
        #出场        
        elif self.CrossoverLo < 0 and self.buytime > 0:            
            self.order = self.sell()            
            self.buytime = 0        
        #止损        
        elif self.datas[0].close < (self.buyprice - 2*self.ATR[0]) and self.buytime > 0:           
            self.order = self.sell()
            self.buytime = 0   
    def stop(self):        
        self.log('(MA Period %2d) Ending Value %.2f' % (self.params.maperiod, self.broker.getvalue()), doprint=True)
        

In [55]:
class SmaCross(bt.Strategy):

    # list of parameters which are configurable for the strategy

    params = dict(

        pfast=5,  # period for the fast moving average

        pslow=50
        
    )
        
    def log(self, txt, dt=None):

        ''' Logging function for this strategy'''

        dt = dt or self.datas[0].datetime.date(0)

        print('%s, %s' % (dt.isoformat(), txt))
        
    def __init__(self):

        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average

        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average

        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

        self.dataclose = self.datas[0].close

        self.order = None

    def next(self):

        self.log('Close,%.2f' % self.dataclose[0])

        print(len(self))

        print(self.order)

        print(self.position)

        if not self.position:  # not in the market

            if self.crossover > 0:  # if fast crosses slow to the upside

                self.order = self.buy()  # enter long

                self.log('-----buy, close price is: %.2f' % self.dataclose[0])

 

        elif self.crossover < 0:  # in the market & cross to the downside

            self.order = self.close()  # close long position

            self.log('-----sell, close price is:%.2f' % self.dataclose[0])
        

In [41]:
###网上找到的 SSA 因子


class ssa_index_ind(bt.Indicator):
    lines = ('ssa',)
    def __init__(self, ssa_window):
        self.params.ssa_window = ssa_window
        # 这个很有用，会有 not maturity生成
        self.addminperiod(self.params.ssa_window * 2)
 
    def get_window_matrix(self, input_array, t, m):
        # 将时间序列变成矩阵
        temp = []
        n = t - m + 1
        for i in range(n):
            temp.append(input_array[i:i + m])
        window_matrix = np.array(temp)
        return window_matrix
 
    def svd_reduce(self, window_matrix):
        # svd分解
        u, s, v = np.linalg.svd(window_matrix)
        m1, n1 = u.shape
        m2, n2 = v.shape
        index = s.argmax()  # get the biggest index
        u1 = u[:, index]
        v1 = v[index]
        u1 = u1.reshape((m1, 1))
        v1 = v1.reshape((1, n2))
        value = s.max()
        new_matrix = value * (np.dot(u1, v1))
        return new_matrix
 
    def recreate_array(self, new_matrix, t, m):
        # 时间序列重构
        ret = []
        n = t - m + 1
        for p in range(1, t + 1):
            if p < m:
                alpha = p
            elif p > t - m + 1:
                alpha = t - p + 1
            else:
                alpha = m
            sigma = 0
            for j in range(1, m + 1):
                i = p - j + 1
                if i > 0 and i < n + 1:
                    sigma += new_matrix[i - 1][j - 1]
            ret.append(sigma / alpha)
        return ret
 
    def SSA(self, input_array, t, m):
        window_matrix = self.get_window_matrix(input_array, t, m)
        new_matrix = self.svd_reduce(window_matrix)
        new_array = self.recreate_array(new_matrix, t, m)
        return new_array
 
    def next(self):
        data_serial = self.data.get(size=self.params.ssa_window * 2)
        self.lines.ssa[0] = self.SSA(data_serial, len(data_serial), int(len(data_serial) / 2))[-1]
 
# Create a Stratey


In [36]:
class NEWStrategy(bt.Strategy):
    params = (
        ('ssa_window', 15),
        ('maperiod', 15),
    )
 
    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
 
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
 
        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
 
        # Add a MovingAverageSimple indicator
        self.ssa = ssa_index_ind(ssa_window=self.params.ssa_window, subplot=False)
        # bt.indicator.LinePlotterIndicator(self.ssa, name='ssa')
        self.sma = bt.indicators.SimpleMovingAverage(period=self.params.maperiod)
    def start(self):
        print("the world call me!")
 
    def prenext(self):
        print("not mature")
 
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
 
        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))
 
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))
 
            self.bar_executed = len(self)
 
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
 
        self.order = None
 
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
 
        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
 
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])
 
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
 
        # Check if we are in the market
        if not self.position:
 
            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.ssa[0]:
 
                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()
 
        else:
 
            if self.dataclose[0] < self.ssa[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
 
                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
    def stop(self):
        print("death")
 


In [ ]:
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(SmaCross)

data = bt.feeds.PandasData(dataname=TX_data,fromdate=datetime.datetime(2016, 12, 1),todate=datetime.datetime(2020, 8, 31))

cerebro.adddata(data)

    # 设置资金
cerebro.broker.setcash(1000000.0)
    # 设置每笔交易交易的股票数量
#cerebro.addsizer(bt.sizers.FixedSize, stake=200)


cerebro.addsizer(TestSizer)   
    # 设置手续费
cerebro.broker.setcommission(commission=0.01)
    # 输出初始资金
print('Starting Portfolio Value:%.2f'  % cerebro.broker.getvalue())
    # 运行策略
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
%matplotlib inline
cerebro.plot()
